In [5]:
import json

In [2]:
import importlib.util
import sys

In [4]:
# Importing LODlitParser AAT and bows modules 
# this code is taken from
# https://stackoverflow.com/questions/67631/how-can-i-import-a-module-dynamically-given-the-full-path

# aat
spec_aat = importlib.util.spec_from_file_location("LODlitParser.aat", "/Users/anesterov/reps/LODlit/LODlitParser/aat.py")
aat = importlib.util.module_from_spec(spec_aat)
sys.modules["LODlitParser.aat"] = aat
spec_aat.loader.exec_module(aat)

# bows
spec_bows = importlib.util.spec_from_file_location("LODlitParser.bows", "/Users/anesterov/reps/LODlit/LODlitParser/bows.py")
bows = importlib.util.module_from_spec(spec_bows)
sys.modules["LODlitParser.bows"] = bows
spec_bows.loader.exec_module(bows)

In [ ]:
# changing the aat function

In [6]:
with open('/Users/anesterov/reps/LODlit/AAT/aat_query_results_en.json','r') as jf:
    aat_search_results = json.load(jf)

In [11]:
for hit in aat_search_results["batavia"]:
    
    literals = []
        
    literals.append(hit["prefLabel"])
    literals.append(hit["prefLabel_comment"])
    literals.append(hit["scopeNote"])
    literals.extend(hit["altLabel"])
    literals.extend(hit["altLabel_comment"])

In [15]:
bows.make_bows(literals,"en",merge_bows=True)

['batavia',
 'dammar',
 'general',
 'term',
 'referring',
 'various',
 'dammar',
 'resin',
 'formerly',
 'shipped',
 'tradename',
 'batavia',
 'jakarta',
 'formerly',
 'batavia',
 'indonesia',
 'general',
 'term',
 'referring',
 'various',
 'dammar',
 'resin',
 'formerly',
 'shipped',
 'tradename',
 'batavia',
 'jakarta',
 'formerly',
 'batavia',
 'indonesia',
 'batavia',
 'damar',
 'dammar',
 'batavia']

In [66]:
all_docs = []
all_tokens = []
    
for value in aat_search_results.values():
    for hit in value:
        if hit["scopeNote"] != "" and hit["scopeNote"] not in all_docs:
            scopeNote_bow = bows.make_bows([hit["scopeNote"]],"en",merge_bows=True)
            all_docs.append(scopeNote_bow)
            # collecting all unique tokens
            for token in scopeNote_bow:
                if token not in all_tokens:
                    all_tokens.append(token)

print(f"Unique tokens: {len(all_tokens)}")    

Unique tokens: 16089


In [98]:
all_docs[0]

['general',
 'term',
 'referring',
 'various',
 'dammar',
 'resin',
 'formerly',
 'shipped',
 'tradename',
 'batavia',
 'jakarta',
 'formerly',
 'batavia',
 'indonesia',
 'general',
 'term',
 'referring',
 'various',
 'dammar',
 'resin',
 'formerly',
 'shipped',
 'tradename',
 'batavia',
 'jakarta',
 'formerly',
 'batavia',
 'indonesia']

In [52]:
big_bow = bows.make_bows(all_docs,"en",merge_bows=True)

In [54]:
len(set(big_bow))

16089

In [68]:
# Making a dict with document frequency (DF) scores for every unique token
doc_freq = {}
for token in all_tokens:
    token_count = 0
    for bow in all_docs:
        if token in bow:
            token_count += 1
    doc_freq[token] = token_count

In [91]:
doc_freq["general"]

195

In [147]:
test_note = aat_search_results["aboriginal"][20]["scopeNote"]

In [148]:
bowed = bows.make_bows([test_note],"en",merge_bows=True)

In [149]:
bowed

['general',
 'designation',
 'race',
 'ethnicity',
 'style',
 'culture',
 'heritage',
 'people',
 'sub',
 'saharan',
 'african',
 'origin',
 'descent',
 'place',
 'outside',
 'africa',
 'member',
 'dark',
 'skinned',
 'group',
 'people',
 'including',
 'australian',
 'aboriginal',
 'origin',
 'descent']

In [150]:
bows.get_top_tokens_tfidf(bowed,doc_freq,len(all_docs))

['descent',
 'origin',
 'saharan',
 'designation',
 'skinned',
 'heritage',
 'sub',
 'australian',
 'outside',
 'ethnicity']

In [64]:
for hit in aat_search_results["inuit"]:
    modified_note = list(set([sent.lstrip(' ') for sent in hit["scopeNote"].split('.') if sent != '']))
    print(modified_note)

['Inuksuit are constructed to act as waypoints, message centers, spiritual veneration, or as hunting decoys', 'Groups of stacked stones or boulders in a figural form, constructed by Inuit individuals', 'The oldest inuksuit date from 2400 to 1800 years ago']
['', 'Often includes a lid topped with a carved, ivory finial; these finials are usually figurative, taking the forms of various Arctic animals', 'The first baleen baskets were woven by Inuit men in the 1910s following tourist interest', 'Baskets constructed of baleen that is coiled around an ivory base']
[]
[]
['', 'Often decorated with beadwork and fringe detailing', 'Parkas characterized by broad shoulders and case on the back meant to carry an infant; created and worn by Inuit communities in North America']
[]
[]
[]
['Sometimes a bead or circular depression is placed in the middle of the paw shape, relating to pierced hand designs common in Alaska Native and Inuit material culture', 'Often, claws of other animals such as seals a

In [ ]:
def get_bows(path_to_results:str, lang:str) -> dict:
    '''
    Getting bag of words (BoW) from the AAT search results for every search term
    path_to_results: str, a path to the search results (in json format)
    lang: str, 'en' or 'nl'
    Returns a dict with BoWs per hit per term: {term:[{aat_URI:['token1','token2','token3']}]}
    '''
    
    wnl = WordNetLemmatizer()
    all_bows = {}
   
    with open(path_to_results,'r') as jf:
        search_results = json.load(jf)

    for query_term, results in search_results.items():

        list_by_term = []

        for hit in results:
            q_bag = {}
            literals = []
            
            if hit["prefLabel"] != "":
                literals.append(hit["prefLabel"]) 
            if hit["prefLabel_comment"] != "":
                literals.append(hit["prefLabel_comment"])
            if hit["scopeNote"] != "":
                literals.append(hit["scopeNote"])
            # altLabel and altLabel comments are lists
            if hit["altLabel"] != []:
                literals.extend(hit["altLabel"])
            if hit["altLabel_comment"] != "":
                literals.extend(hit["altLabel_comment"])

            bow = []
            for lit in literals:
                bow.extend(lit.replace('(','').replace(')','').replace('-',' ').replace('/',' ')\
                           .replace(',','').lower().split(' '))

            # checking lang for stopwords
            if lang == 'en':
                bag_filtered = [wnl.lemmatize(w) for w in bow if w not in stopwords.words('english') \
                                    and re.search('(\W|\d)',w) == None and w != '']
            if lang == 'nl':
                bag_filtered = [simplemma.lemmatize(w,lang='nl') for w in bow if w not in stopwords.words('dutch') \
                                    and re.search('(\W|\d)',w) == None and w != '']

            q_bag[hit["aat_uri"]] = bag_filtered

            list_by_term.append(q_bag)

        all_bows[query_term] = list_by_term

    return all_bows

In [ ]:
def get_unique_tokens_and_docs(source:dict) -> tuple:
    '''
    Gets unique tokens and documents in a file
    source: dict
    Prints N of unique tokens
    Returns a tuple, where 0: list of tokens (str), 1: list of documents (list)
    '''
    all_docs = []
    all_tokens = []
    
    for value in source.values():
        # taking only unique bows
        for bow in value["bow"]:
            if bow not in all_docs:
                all_docs.append(bow)
                # collecting all unique tokens
                for token in bow:
                    if token not in all_tokens:
                        all_tokens.append(token)
    
    print(f"Unique tokens: {len(all_tokens)}")
    
    tokens_docs = (all_tokens, all_docs) 
    
    return tokens_docs